<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Inżynieria-lingwistyczna" data-toc-modified-id="Inżynieria-lingwistyczna-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Inżynieria lingwistyczna</a></span></li><li><span><a href="#Zadanie-domowe-2" data-toc-modified-id="Zadanie-domowe-2-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Zadanie domowe 2</a></span><ul class="toc-item"><li><span><a href="#Zadanie-1---eksploracja-przestrzeni-zagnieżdżeń" data-toc-modified-id="Zadanie-1---eksploracja-przestrzeni-zagnieżdżeń-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Zadanie 1 - eksploracja przestrzeni zagnieżdżeń</a></span></li><li><span><a href="#Zadanie-2---zagnieżdżenia-dokumentów" data-toc-modified-id="Zadanie-2---zagnieżdżenia-dokumentów-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Zadanie 2 - zagnieżdżenia dokumentów</a></span></li></ul></li><li><span><a href="#Zadanie-3---konstruowanie-zagnieżdżeń" data-toc-modified-id="Zadanie-3---konstruowanie-zagnieżdżeń-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Zadanie 3 - konstruowanie zagnieżdżeń</a></span></li></ul></div>

# Inżynieria lingwistyczna
Ten notebook jest oceniany półautomatycznie. Nie twórz ani nie usuwaj komórek - struktura notebooka musi zostać zachowana. Odpowiedź wypełnij tam gdzie jest na to wskazane miejsce - odpowiedzi w innych miejscach nie będą sprawdzane (nie są widoczne dla sprawdzającego w systemie).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

---

# Zadanie domowe 2

In [1]:
import pandas as pd
import numpy as np
       

## Zadanie 1 - eksploracja przestrzeni zagnieżdżeń
Wczytajmy do przestrzeni plik zagnieżdżeń, który należy pobrać ze strony:
https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.pl.vec Są to zagnieżdzenia dla języka polskiego uzyskane systemem fastText.

Do przestrzeni wczytujemy tylko 100 tys. najczęstrzych słów, tak aby operacje przebiegały szybciej.

In [2]:
import io
def load_vectors(fname, limit = 100000):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    n = min(n,limit)
    embeddings = np.empty((n,d), dtype = np.float)
    words_idx = []
    for i, line in enumerate(fin):
        if i >= limit:
            break
        tokens = line.rstrip().split(' ')
        words_idx.append(tokens[0])
        embeddings[i] =  np.array(tokens[1:]).astype(np.float)
    return words_idx, embeddings
words_idx, embeddings = load_vectors('wiki.pl.vec')

Poniższe zadania mają na celu poekserymentowanie z przestrzenią zagnieżdżeń, ale też zrozumienie stojącymi za nich operacji. Dozwolone jest korzystanie tylko z podstawowych operatorów Python i numpy (w szczególności zakaz dotyczy: sklearn, gensim, fasttext itd.)

Jeśli potrzebujesz do dalszego przetwarzania przeprowadzenia jakichś normalizacji macierzy -- możesz wstępnie przetworzyć macierz zagnieżdzeń poniżej. Pamiętaj, że sprawdzarka będzie używała wywołań do `embeddings` (i `words_idx`) -- musisz nadpisać macierz zagnieżdzeń. To pole jest pomocnicze i nie podlega ocenie.

In [3]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

Zaimplementuj funkcję, która obliczy podobieństwo kosinusowe pomiędzy dwoma wyrazami.

In [4]:
def calc_sim(word_a, word_b, words_idx, embeddings):
    embedding_a = embeddings[words_idx.index(word_a)]
    embedding_b = embeddings[words_idx.index(word_b)]
    return np.dot(embedding_a, embedding_b) / (np.linalg.norm(embedding_a) * np.linalg.norm(embedding_b))

In [5]:
from nose.tools import assert_almost_equal
assert_almost_equal(calc_sim("bieber", "rihanna", words_idx, embeddings), calc_sim("rihanna", "bieber", words_idx, embeddings))

Policz podobieństwo pomiędzy wyrazem `bieber` a wyrazami:
    - `rihanna`
    - `piłsudski`
    - `kanada`
    - `polska`
    - `piosenka`

In [6]:
calc_sim("bieber", "rihanna", words_idx, embeddings)

0.524583248263655

In [7]:
# YOUR CODE HERE
words_ = ["rihanna", "piłsudski", "kanada", "polska", "piosenka"]
[calc_sim('bieber', word, words_idx, embeddings) for word in words_]

[0.524583248263655,
 0.19303958051463557,
 0.20042742126487925,
 0.12505934735679378,
 0.2874871368858332]

Zaimplementuj funkcję, która zwróci najbardziej podobne słowa (miara kosinusowa) do danego słowa `word`. W wyniku wypisz tylko `top` słów z najbliższymi zagnieżdżeniami, pomijając słowo `word`.

In [45]:
def remove_word_from_list(list_, word):
    if word in list_: list_.remove(word)
    return list_

def find_similar_from_embedding(embedding_a, words_idx, embedding_matrix, top):
    tiled_embedding_a = np.tile(embedding_a, len(words_idx)).reshape((len(words_idx), -1)) 
    embedding_a_length = np.linalg.norm(embedding_a)
    embedding_matrix_lenghts = np.linalg.norm(embedding_matrix, axis=1)
    similarities = np.einsum('ij,ij->i', tiled_embedding_a, embedding_matrix) / (embedding_matrix_lenghts * embedding_a_length)
    top_similar = np.argsort(similarities)[-top:]
    return [words_idx[i] for i in np.flipud(top_similar)]

def find_similar(word, words_idx, embedding_matrix, top=10):
    embedding_a = embedding_matrix[words_idx.index(word)]
    word_list = find_similar_from_embedding(embedding_a, words_idx, embedding_matrix, top + 1)
    return remove_word_from_list(word_list, word)[:top]


In [12]:
assert len(find_similar("radość", words_idx, embeddings)) == 10

Znajdź najbardziej podobne słowa do kobieta, politechnika, mateusz, szczecin, niemcy, piłsudski

In [46]:
find_similar("kobieta", words_idx, embeddings)


['kobietą',
 'dziewczyna',
 'mężczyzna',
 'kobietę',
 'dziewczynka',
 'mężczyznę',
 'staruszka',
 'mężczyzną',
 'kobiecie',
 'mężczyzny']

In [19]:
# YOUR CODE HERE
words_ = ["kobieta", "politechnika", "mateusz", "szczecin", "niemcy", "piłsudski"]
[find_similar(word, words_idx, embeddings) for word in words_]

[['kobietą',
  'dziewczyna',
  'mężczyzna',
  'kobietę',
  'dziewczynka',
  'mężczyznę',
  'staruszka',
  'mężczyzną',
  'kobiecie',
  'mężczyzny'],
 ['politechniki',
  'politechniką',
  'politechnikę',
  'politechniczny',
  'politechnice',
  'politechnicznej',
  'politechnicznego',
  'politechnicznym',
  'inżynierska',
  'elektrotechnika'],
 ['łukasz',
  'bartłomiej',
  'bartosz',
  'kacper',
  'marcin',
  'mateusza',
  'tomasz',
  'patryk',
  'rafał',
  'mateuszem'],
 ['szczecinek',
  'szczeciński',
  'szczecinem',
  'gryfino',
  'szczecinie',
  'stargard',
  'szczecina',
  'koszalin',
  'szczecińska',
  'świnoujście'],
 ['niemieccy',
  'naziści',
  'alianci',
  'okupanci',
  'polacy',
  'hitlerowcy',
  'niemieckie',
  'rosjanie',
  'niemców',
  'niemcom'],
 ['piłsudskim',
  'piłsudskiego',
  'piłsudskiemu',
  'sosnkowski',
  'mościcki',
  'śmigły',
  'józef',
  'żeligowski',
  'piłsudczyków',
  'sosnkowskiego']]

Krótko skomentuj wyniki dla słowa `niemcy`. Które z powstałych analogii biorą się z semantycznego powiązania a które z semantycznego podobieństwa?

Uzyskany słowa dla słowa niemcy to:
'niemieccy', 'naziści', 'alianci', 'okupanci', 'polacy', 'hitlerowcy', 'niemieckie', 'rosjanie', 'niemców', 'niemcom'

Większość słów to jakieś warianty słowa niemcy (odmiana lub pochodne przemitoniki), jednakże można tutaj zauważyć takie słowa jak okupanci itp. Może to wynikać z tego że zagnieżdżenia zostały uzyskane na polskiej wikipedii, na której jest względnie dużo informacji na temat rozbiorów i wojny, przez co nasze zagnieżdżenia posiadają pewien bias na znaczenia tego słowa wojenną.

Semantyczne powiązania -> 'niemieccy', 'niemieckie', 'niemców', 'niemcom'
Semantyczne podobieństwo -> 'naziści', 'alianci', 'okupanci','hitlerowcy'


Zaimplementuj funkcje szukającą brakującego elementu relacji ,,`word_a` jest do `word_a2` jak `word_b` jest do...''. Funkcja powinna zwrócić 10 najbardziej pasujących słow z pominięciem słów będących jej argumentami.

In [26]:
def find_similar_pair(word_a, word_a2, word_b,  words_idx, matrix, top=10):
    embedding_a, embedding_a_2 = matrix[words_idx.index(word_a)],  matrix[words_idx.index(word_a2)]
    dif_a = embedding_a_2 - embedding_a
    embedding_b = matrix[words_idx.index(word_b)]
    embedding_b2 = (embedding_b + dif_a)
    
    most_similar_words = find_similar_from_embedding(embedding_b2, words_idx, matrix, top + 3)
    remove_word_from_list(most_similar_words, word_a)
    remove_word_from_list(most_similar_words, word_a2)
    remove_word_from_list(most_similar_words, word_b)
    return most_similar_words[:top]


In [27]:
assert find_similar_pair( "mężczyzna", "król", "kobieta", words_idx, embeddings)[0] == "królowa"

Pieniądze są do profesora jak wiedza do...

In [28]:
find_similar_pair( "pieniądze", "profesora", "wiedza", words_idx, embeddings)

['habilitowanego',
 'habilitacja',
 'docenta',
 'profesorski',
 '„wiedza',
 'habilitację',
 'habilitowany',
 'habilitowała',
 'habilitowanych',
 'profesorem']

Mateusza jest do mateusz jak łukasza do ...

In [29]:
find_similar_pair( "mateusza", "mateusz", "łukasza", words_idx, embeddings)

['łukasz',
 'bartłomiej',
 'bartosz',
 'maciej',
 'tomasz',
 'rafał',
 'patryk',
 'marcin',
 'michał',
 'przemysław']

Warszawa jest do "polska" jak "berlin" do ...

In [30]:
find_similar_pair( "warszawa", "polska", "berlin", words_idx, embeddings)

['niemiecka',
 'berliner',
 'wschodnioniemiecka',
 'berlińska',
 'deutschland',
 'deutsche',
 'brandenburg',
 'berlinem',
 'germany',
 'niemcy']

Zurich jest do ETH jak Poznań do ...

In [31]:
find_similar_pair( "zurich", "eth", "poznań", words_idx, embeddings)

['„poznań',
 'wrocław',
 'poznania',
 'poznańskie',
 'uam',
 'poznaniu',
 'kraków',
 'gniezno',
 'poznańską',
 'wlkp']

Niemcy są do Merkel jak Polska do ...

In [32]:
find_similar_pair( "niemcy", "merkel", "polska", words_idx, embeddings)

['kaczyńska',
 'lewandowska',
 'kwaśniewska',
 'ekonomistka',
 'lekarka',
 'parlamentarzystka',
 'marcinkiewicz',
 'olszewska',
 'bezpartyjna',
 'dziennikarka']

Na wektorach możemy wykonywać standardowe operacje algebry liniowej takie jak np. projekcja czyli rzutowanie danych na jakichś zbiór osi (więcej: notatki z algebry liniowej np. https://ocw.mit.edu/courses/mathematics/18-06sc-linear-algebra-fall-2011/least-squares-determinants-and-eigenvalues/projections-onto-subspaces/). W szczególności może to się przydać do zrzutowania słowa na przestrzeń w której pewny wybrany kierunek (wskazywany przez wektor) jest eliminowany.

Do czego może to się przydać? Jeśli uruchomisz funkcję `find_similar` dla słowa ,,mateusza'' znajdziesz m.in. ,,łukasza'' ale także ,,ewangelia'', ,,ewangelisty'' i ,,apostoła''. Chcąc pominąc kontekst religijny tego słowa możesz zrzutować jego reprezentacje na przestrzeń bez wektora ,,ewangelia'' i poszukać jego najbliższych sąsiadów (którymi będą teraz po prostu imiona męskie). Zaimplementuj taką funkcję.


In [42]:
def projection(v_a, v_b):
    return v_a * (v_a.T.dot(v_b) / v_a.T.dot(v_a))

def remove_word(emb_a, rem_word, words_idx, embedding_matrix):
    emb_b = embedding_matrix[words_idx.index(rem_word)]
    proj = projection(emb_b, emb_a)
    return emb_a - proj

def find_similar_with_rejection(word, remove, words_idx, matrix, top=10):
    """
    Działanie analogiczne do find_similar z dodatkowym parametrem remove, 
    który jest *listą* słów, które należy wyrzucić poprzez projekcję.
    Dla remove=[] powinno się zwracać dokładnie to samo co find_similar
    """
    embedding = matrix[words_idx.index(word)]
    for rem in remove:
        old = embedding
        embedding = remove_word(embedding, rem, words_idx, matrix)
    word_list = find_similar_from_embedding(embedding, words_idx, matrix, top + 1)
    return remove_word_from_list(word_list, word)[:top]
    
    
print ("Standardowe poszukiwanie:", find_similar_with_rejection("mateusza",[] , words_idx, embeddings))
print ("Poszukiwanie po projekcji:", find_similar_with_rejection("mateusza",["ewangelia"] , words_idx, embeddings))

Standardowe poszukiwanie: ['łukasza', 'ewangelii', 'ewangelisty', 'ewangelia', 'bartłomieja', 'ewangeliach', 'apostoła', 'mateusz', 'tymoteusza', 'jakuba']
Poszukiwanie po projekcji: ['macieja', 'bartłomieja', 'marcina', 'łukasza', 'andrzeja', 'piotra', 'jakuba', 'antoniego', 'michała', 'tomasza']


In [41]:
assert "ewangelii" in find_similar_with_rejection("mateusza",[] , words_idx, embeddings)
assert "ewangelii" not in find_similar_with_rejection("mateusza",["ewangelia"] , words_idx, embeddings)
assert "ewangelisty" not in find_similar_with_rejection("mateusza",["ewangelia"] , words_idx, embeddings)


Analogicznie słowo ,,java'' jest nie tylko nazwą języka programownia (https://pl.wikipedia.org/wiki/Java_(ujednoznacznienie)) -- jest np. nazwą geograficzną (indonezyjska wyspa koło Sumatry). Sprawdź jakie wyrazy są podobne do "java" oraz po odrzuceniu kierunku "javascript" (tj. kierunku związanego z językami programowania).

In [51]:
print ("Standardowe poszukiwanie:", find_similar_with_rejection("java",[] , words_idx, embeddings))
print ("Poszukiwanie po projekcji:", find_similar_with_rejection("java",["javascript"] , words_idx, embeddings))
print ("Poszukiwanie po projekcji:", find_similar_with_rejection("java",["javascript", "python"] , words_idx, embeddings))
print ("Poszukiwanie po projekcji:", find_similar_with_rejection("java",["c"] , words_idx, embeddings)) # zagnieżdżenie c nie za bardzo ma odniesienie do języka programowania
print ("Poszukiwanie po projekcji:", find_similar_with_rejection("java",["c++"] , words_idx, embeddings))
print ("Poszukiwanie po projekcji:", find_similar_with_rejection("java",["wyspa"] , words_idx, embeddings))

Standardowe poszukiwanie: ['javascript', 'javy', 'c#', 'c++', 'programowania', 'implementacja', 'lisp', 'framework', 'api', 'programistyczne']
Poszukiwanie po projekcji: ['sumatra', 'tromp', 'krążowniki', 'indonezja', 'niszczycielami', 'penang', 'niszczyciele', 'krążowników', 'hmas', 'jawa']
Poszukiwanie po projekcji: ['sumatra', 'indonezja', 'tromp', 'krążowniki', 'niszczycielami', 'penang', 'flagowy', 'jawa', 'niszczyciele', 'krążowników']
Poszukiwanie po projekcji: ['javy', 'javascript', 'framework', 'api', 'programistyczne', 'programowania', 'implementacja', 'mysql', 'postgresql', 'lisp']
Poszukiwanie po projekcji: ['sumatra', 'javy', 'emden', 'hmas', 'wake', 'broke', 'sandwich', 'navigator', 'yap', 'ajax']
Poszukiwanie po projekcji: ['javascript', 'javy', 'c#', 'c++', 'lisp', 'programowania', 'framework', 'implementacja', 'mysql', 'programistyczne']


Spróbuj poekseprymentować samemu!

In [83]:
print ("Standardowe poszukiwanie:", find_similar_with_rejection("religia",[] , words_idx, embeddings))
print ("Poszukiwanie po projekcji:", find_similar_with_rejection("religia",["chrześcijaństwo"] , words_idx, embeddings))
print ("Poszukiwanie po projekcji:", find_similar_with_rejection("religia",["islam"] , words_idx, embeddings))

Standardowe poszukiwanie: ['religie', 'religii', 'religiach', 'chrześcijaństwo', 'religiami', 'chrześcijaństwie', 'religią', 'kultura', 'chrześcijaństwem', 'religijna']
Poszukiwanie po projekcji: ['kultura', 'antropologia', 'rozrywka', 'socjologiczna', 'cywilizacja', 'edukacja', 'etyka', 'filozofia', 'kulturowa', 'etnografia']
Poszukiwanie po projekcji: ['kultura', 'etyka', 'filozofia', 'cywilizacja', 'antropologia', 'religiach', 'religie', 'religii', 'religijna', 'religiami']


Wykonanie projekcji w przestrzeni zagnieżdżeń może być jedną z prostych technik zwalczenia tzw. gender bias (http://wordbias.umiacs.umd.edu/) w reprezentacji słów. Okazuje się, że wykonanie projekcji macierzy zagnieżdżeń na przestrzeń w której ,,brakuje kierunku he-she'' może być bardzo prostą techniką zredukowania tego typu obciążenia.

## Zadanie 2 - zagnieżdżenia dokumentów
W tym ćwiczeniu powócimy do zbioru tweetów, który analizowaliśmy w poprzednim dokumencie.

In [84]:
from helpers import DataSet
training_set = DataSet(['tweets.txt'])

Reading data set ['tweets.txt']


In [85]:
for i in training_set.tweets:
    print(i.text)
    print(i.tokens)
    print(i.clazz)
    break

dear @Microsoft the newOoffice for Mac is great and all, but no Lync update? C'mon.
['dear', '@microsoft', 'the', 'newooffice', 'for', 'mac', 'is', 'great', 'and', 'all', ',', 'but', 'no', 'lync', 'update', '?', "c'mon", '.']
negative


Tym razem do zbudowania reprezentacji będziemy używać narzędzie Universal Sentence Encoder stworzone przez Googla na bazie głębokiej sieci uśredniającej (i architektur rekurencyjnych). Poniższy kod pokazuje sposób użycia tego narzędzia. 
Kod spokojnie można wywoływać na CPU -- choć ściąganie modelu trochę może potrwać.

In [86]:
import tensorflow as tf
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embeddings = embed([
    "The quick brown fox jumps over the lazy dog.",
    "I am a sentence for which I would like to get its embedding"])
print (embeddings)

ModuleNotFoundError: No module named 'tensorflow'

Wykorzystując reprezetnację USE wytrenuj wybrany klasyfikator z pakietu `sklearn` i zweryfikuj jego jakość działania.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Skomentuj wyniki i odnieś je do wyników z poprzedniego zadania domowego. Na ile użycie reprezentacji rozproszonych pozwoliło na poprawę wyników?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

# Zadanie 3 - konstruowanie zagnieżdżeń
W tym ćwiczeniu kontynuujemy pracę z tweetami, ale pomijamy całkowicie ich klasy. Zbiór tweetów potraktujemy jako korpus do nauczenia zagnieżdżeń słów przy pomocy macierzy PMI.
- Wypełnij macierz kontekst - dokument przy użyciu symetrycznego okna o promieniu 4 (po 4 słowa w każdą stronę)
- Możesz ograniczyć słownictwo do 10K słów
- Przekształć macierz w macierz PPMI
- Stwórz zagnieżdżenia wykorzystując dekompozycję SVD do wybranej wymiarowości $d$ (ze względu na koszt obliczeniowy może to być mała wymiarowość np. $d=10$)

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Przetestuj działanie Twoich zagnieżdżeń wykorzystując funkcję `find_similar` na wybranych słowach.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()